In [1]:
from pathlib import Path
import struct

In [2]:
rootdir = Path('.').absolute().parent
basic_msg_file = rootdir / 'out' / 'data' / 'broken.c.msg'
with open(basic_msg_file, 'rb') as f:
    msg = f.read()

In [8]:
x, y = struct.unpack('ff', msg)

In [9]:
len(msg)

8

In [19]:
msg

b'\x00\x00\x80?\x00\x00\x00@'

In [10]:
xy = [x, y]
struct.pack('ff', *xy)

b'\x00\x00\x80?\x00\x00\x00@'

In [15]:
class TruncatedMessage:
    sizes = {
        'byte': 1,
        'bool': 1,
        'int16': 2,
        'uint16': 2,
        'int32': 4,
        'uint32': 4,
        'int64': 8,
        'uint64': 8,
        'float': 4,
        'double': 8,
    }
    format = 'ff'
    
    def __init__(self, x, y):
        self.x = x
        self.y = y

        self.vars = [x, y]
    
    @property
    def encoded(self):
        return struct.pack(self.format, *self.vars)

    def write_bytes(self, stream, tag):
        stream.write(self.encoded)
        raise NotImplementedError
    
    def get_message_type(self):
        raise NotImplementedError
    
    def get_size_in_bytes(self):
        return self.sizes['float'] * 2
    
    @classmethod
    def from_bytes(self, buffer):
        vals = struct.unpack(self.format, buffer)
        return TruncatedMessage(*vals)


In [16]:
t = TruncatedMessage.from_bytes(msg)

In [17]:
t.x, t.y

(1.0, 2.0)

In [18]:
t.get_size_in_bytes()

8

In [ ]:
class ListMessage:
    sizes = {
        'byte': 1,
        'bool': 1,
        'int16': 2,
        'uint16': 2,
        'int32': 4,
        'uint32': 4,
        'int64': 8,
        'uint64': 8,
        'float': 4,
        'double': 8,
    }

    def __init__(self, ints: list[int]):
        self.ints = ints
    
    @property
    def encoded(self):
        l = len(self.ints) * self.sizes['int16']
        return struct.pack(f'I{l}h', l, *self.ints)
    
    def write_bytes(self, stream, tag):
        stream.write(self.encoded)
        raise NotImplementedError
    
    def get_message_type(self):
        raise NotImplementedError
    
    def get_size_in_bytes(self):
        return self.sizes['int16'] * len(self.ints) + self.sizes['uint32']
    
    @classmethod
    def from_bytes(self, buffer):
        l = struct.unpack('I', buffer.read(4))
        vals = struct.unpack(self.format, buffer)
        return TruncatedMessage(*vals)

In [32]:
data = [1, 2, 3, 4, -5]
l = len(data)
d = struct.pack(f'I{l}h', l * 2, *data)
d

b'\n\x00\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\xfb\xff'

In [33]:
with open('dumbtest.bin', 'wb') as f:
    f.write(d)

In [39]:
with open('dumbtest.bin', 'rb') as f:
    l = struct.unpack('I', f.read(4))[0]
    print(f'There are {l} bytes of data')
    dbytes = f.read(l)
    data = list(struct.unpack(f'{int(l/2)}h', dbytes))
    print(data)

There are 10 bytes of data
[1, 2, 3, 4, -5]
